# Lazy Sequences

### Sequences Without End


In [1]:
;; A simple sequence

(def jack "All work and no play makes Jack a dull boy.")

(def text [jack jack jack jack jack jack jack jack jack jack])

#'user/text

In [2]:
;; Using 'repeat' to generate items like the last sequence 

(def repeated-text (repeat jack))


#'user/repeated-text

In [3]:
;; The 1st item

(first repeated-text)


"All work and no play makes Jack a dull boy."

In [4]:
;; The 10th item

(nth repeated-text 10)


"All work and no play makes Jack a dull boy."

In [5]:
;; The 10202th item (!)

(nth repeated-text 10202) ; An example of a lazy sequence


"All work and no play makes Jack a dull boy."

## More Interesting Laziness


In [6]:
;; A lazy sequence created with 'cycle'

(take 7 (cycle [1 2 3]))


(1 2 3 1 2 3 1)

In [7]:
;; A lazy sequence created with 'iterate'

(def numbers (iterate inc 1))


#'user/numbers

In [8]:
(nth numbers 0) ; 1st item


1

In [9]:
(nth numbers 1)  ; 2nd item


2

In [10]:
(nth numbers 99)  ; 100th item


100

### Lazy Friends

In [11]:
;; This sequence doesn't create 1000000000 items immediately

(def many-nums (take 1000000000 (iterate inc 1)))


#'user/many-nums

In [12]:
;; THis will take the first 20 items, not the first 1000000000

(println (take 20 (take 1000000000 (iterate inc 1))))


(1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20)


nil

In [13]:
;; The 'map' function is also lazy

(def evens 
    (map #(* 2 %) (iterate inc 1)))

(take 20 evens)


(2 4 6 8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40)

In [14]:
;; The 'interleave' function is also lazy

(take 10 (interleave numbers evens))


(1 2 2 4 3 6 4 8 5 10)

### Laziness in Practice

In [15]:
;; We can create fake books data manually as follows

(def test-books
    [{:author "Bob Jordan", :title "Wheel of Time, Book 1"}
    {:author "Jane Austen", :title "Wheel of Time, Book 2"}
    {:author "Chuck Dickens", :title "Wheel of Time, Book 3"}
    {:author "Leo Tolstoy", :title "Wheel of Time, Book 4"}
    {:author "Bob Poe", :title "Wheel of Time, Book 5"}
    {:author "Jane Jordan", :title "Wheel of Time, Book 6"}
    {:author "Chuck Austen", :title "Wheel of Time, Book 7"}])


#'user/test-books

In [16]:
;; But we can generate infinite book titles as follows

(def numbers (iterate inc 1))

(def titles 
    (map #(str "Wheel of Time, Book " % ) numbers))

#'user/titles

In [17]:
;; Also, we can generate author names as follows

(def first-names ["Bob" "Jane" "Chuck" "Leo"])
(def last-names ["Jordan" "Austen" "Dickens" "Tolstoy" "Poe"])

(defn combine-names [fname lname]
    (str fname " " lname))

(def authors
    (map combine-names (cycle first-names) (cycle last-names)))

(defn make-book [title author]
    {:author author :title title})

#'user/make-book

In [18]:
(def test-books (map make-book titles authors))

(take 2 test-books)


({:author "Bob Jordan", :title "Wheel of Time, Book 1"} {:author "Jane Austen", :title "Wheel of Time, Book 2"})

### Behind the Scenes


In [19]:
;; The 'lazy-seq' function turns an expression into a lazy sequence

(defn chatty-vector []
    (println "Here we go!") [1 2 3])

(def s (lazy-seq (chatty-vector))) ; Nothing is computed yet


#'user/s

In [20]:
(take 2 s) ; Only when needed the sequence is computed



(Here we go!
1 2)

In [21]:
;; A custom version of 'repeat' can be implemented as follows

(defn my-repeat [x]
    (cons x (lazy-seq (my-repeat x))))

(take 3 (my-repeat 6))


(6 6 6)

In [22]:
;; A custom version of 'iterate' can be implemented as follows

(defn my-iterate [f x]
    (cons x (lazy-seq (my-iterate f (f x)))))

(take 3 (my-iterate #(* 2 %) 6))


(6 12 24)

In [25]:
;; A custom version of 'map' can be implemented as follows

(defn my-map [f col]
    (when-not (empty? col)
        (cons (f (first col))
                 (lazy-seq (my-map f (rest col))))))

(my-map #(* 3 %) [1 2 3])


(3 6 9)

### Issues with lazy sequences

In [26]:
;; Don't evaluate infinite lazy sequences in the REPL

(def numbers (iterate inc 1))

"""
(println numbers) ; NO!
"""

""

In [28]:
;; Insteat, set the *print-length* var to a finite value

(set! *print-length* 10)

(println numbers)


(1 2 3 4 5 6 7 8 9 10 ...)


nil

In [35]:
;; A lazy sequence can be evaluated with 'doall'

(def chapters 
    (take 10 (map slurp (map #(str "chap" % ".txt") numbers)))) ; 'slurp' get the contents of a file as a string

; (doall chapters)


#'user/chapters

In [34]:
;; A lazy sequence can also be evaluated with 'doseq'

(doseq [c chapters]
    ;(println "The chapter text is" c)
    )

nil